In [ ]:
from tqdm.notebook import tqdm
from database_tools.io.records import generate_record_paths, get_header_record
from database_tools.filtering.exclusion import header_has_signals

In [ ]:
signals = ['PLETH', 'ABP']
min_length = 75000  # 10 min in samples (125 Hz)

valid_segs = []
for path in tqdm(generate_record_paths(name='adults')):
    layout = get_header_record(path=path, record_type='layout')
    if layout is None: continue
    if not header_has_signals(layout, signals): continue

    master = get_header_record(path=path, record_type='data')
    if master is None: continue

    segments = []
    for seg_name, n_samples in zip(master.seg_name, master.seg_len):
        if (n_samples > min_length) & (seg_name != '~'):
            seg_path = path + '/' + seg_name
            hea = get_header_record(path=seg_path, record_type='data')
            if hea is None: continue
            if header_has_signals(hea, signals):
                segments.append(seg_path)
    valid_segs += segments
    if len(valid_segs) > 100:
        break